In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [3]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [4]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [8]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config
config.target = "energy"

In [10]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [11]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=False)
model.target_scaler = scaler

In [12]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 78s 3s/step - loss: 0.9636


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 6.51284, saving model to callback/val_mae_00001_6.512837.hdf5


Epoch 2/100
21/21 [==============================] - 55s 3s/step - loss: 0.1179


...

Epoch 26/100
21/21 [==============================] - 55s 3s/step - loss: 0.0546
Epoch 27/100
21/21 [==============================] - 55s 3s/step - loss: 0.0473
Epoch 28/100
21/21 [==============================] - 55s 3s/step - loss: 0.0519
Epoch 29/100
21/21 [==============================] - 55s 3s/step - loss: 0.0527
Epoch 30/100
21/21 [==============================] - 55s 3s/step - loss: 0.0553
Epoch 31/100
21/21 [==============================] - 55s 3s/step - loss: 0.0421
Epoch 32/100
21/21 [==============================] - 55s 3s/step - loss: 0.0616
Epoch 33/100
21/21 [==============================] - 55s 3s/step - loss: 0.0387
Epoch 34/100
21/21 [==============================] - 55s 3s/step - loss: 0.0568
Epoch 35/100
21/21 [==============================] - 55s 3s/step - loss: 0.0372
Epoch 36/100
21/21 [==============================] - 55s 3s/step - loss: 0.0378
Epoch 37/100
21/21 [==============================] - 55s 3s/step - loss: 0.0524
Epoch 38/100
21/21 [========

INFO:megnet.callbacks:
Epoch 00038: val_mae improved from 2.47856 to 2.41978, saving model to callback/val_mae_00038_2.419784.hdf5


Epoch 39/100
21/21 [==============================] - 55s 3s/step - loss: 0.0469
Epoch 40/100
21/21 [==============================] - 55s 3s/step - loss: 0.0550
Epoch 41/100
21/21 [==============================] - 55s 3s/step - loss: 0.0375
Epoch 42/100
21/21 [==============================] - 56s 3s/step - loss: 0.0481
Epoch 43/100
21/21 [==============================] - 55s 3s/step - loss: 0.0504
Epoch 44/100
21/21 [==============================] - 55s 3s/step - loss: 0.0440
Epoch 45/100
21/21 [==============================] - 55s 3s/step - loss: 0.0417
Epoch 46/100
21/21 [==============================] - 55s 3s/step - loss: 0.0542
Epoch 47/100
21/21 [==============================] - 55s 3s/step - loss: 0.0439
Epoch 48/100
21/21 [==============================] - 55s 3s/step - loss: 0.0555
Epoch 49/100
21/21 [==============================] - 55s 3s/step - loss: 0.0739
Epoch 50/100
21/21 [==============================] - 55s 3s/step - loss: 0.0607
Epoch 51/100
21/21 [========

INFO:megnet.callbacks:
Epoch 00076: val_mae improved from 2.41978 to 2.36485, saving model to callback/val_mae_00076_2.364853.hdf5


Epoch 77/100
21/21 [==============================] - 55s 3s/step - loss: 0.0703
Epoch 78/100
21/21 [==============================] - 56s 3s/step - loss: 0.0646
Epoch 79/100
21/21 [==============================] - 56s 3s/step - loss: 0.0468
Epoch 80/100
21/21 [==============================] - 55s 3s/step - loss: 0.0781
Epoch 81/100
21/21 [==============================] - 55s 3s/step - loss: 0.0517
Epoch 82/100
21/21 [==============================] - 55s 3s/step - loss: 0.0634
Epoch 83/100
21/21 [==============================] - 56s 3s/step - loss: 0.0633
Epoch 84/100
21/21 [==============================] - 55s 3s/step - loss: 0.0473
Epoch 85/100
21/21 [==============================] - 56s 3s/step - loss: 0.0572
Epoch 86/100
21/21 [==============================] - 55s 3s/step - loss: 0.0440
Epoch 87/100
21/21 [==============================] - 55s 3s/step - loss: 0.0481
Epoch 88/100
21/21 [==============================] - 55s 3s/step - loss: 0.0318
Epoch 89/100
21/21 [========